# Conda Meta recipe for `node` Stable, `npm` latest

In [4]:
import os
from hashlib import sha256
from requests import get
from urllib.parse import urljoin as join
import re
import yaml
import jinja2
from PIL import Image

In [1]:
DIST = "https://nodejs.org/dist"
PLATFORMS = [
    ["linux", "64", "linux64"],
    ["linux", "86", "linux32"],
    ["darwin", "64", "osx"]
]

In [8]:
all_dists = get(DIST).content.decode()

In [23]:
STABLE_V = sorted(re.findall(r"v4\.\d+\.\d+", all_dists))[-1]
STABLE = STABLE_V[1:]
STABLE_V, STABLE

('v4.2.6', '4.2.6')

In [25]:
shas_txt = get("/".join([DIST, STABLE_V, "SHASUMS256.txt"])).content.decode().strip()

In [27]:
SHAS = dict([list(reversed(it.split(r"  "))) for it in shas_txt.split("\n")])
len(SHAS)

27

In [ ]:
from glob import glob
 
jpgfiles = glob('*.jpg')
for u in jpgfiles:
    out = u.replace('jpg','eps')
    print "Converting %s to %s" % (u, out)
    img=Image.read(u)
    img.thumbnails((800,800)) # Changing the size
    img.save(out)

In [2]:
TMPL = """
{% macro fname(platform, arch) %}node-{{ STABLE_V }}-{{ platform }}-x{{ arch }}.tar.gz{% endmacro %}
package:
  name: nodejs
  version: {{ STABLE }}

build:
  number: 0

about:
  home: https://nodejs.org
  license: MIT

source:
  {% for platform, arch, conda_platform in PLATFORMS %}
  fn: node-v{{ STABLE }}-{{ platform }}-x{{ arch }}.tar.gz # [linux64]
  url: {{ DIST }}/{{ STABLE_V }}/{{ fname(platform, arch) }} # [{{ conda_platform }}]
  sha256: {{ SHAS[fname(platform, arch)] }} # [{{ conda_platform }}]
  {% endfor %}

requirements:
  build:
    - python # [win]

test:
  commands:
    - node --version
    - npm --version
    - npm config get prefix -g
    - npm install babel babel-core babel-preset-es2015 casperjs gulp istanbul jupyter-js-services karma less phantomjs
"""

In [59]:
with open(os.path.join("conda.recipe", "meta.yaml"), "w+") as meta:
    meta.write(jinja2.Template(TMPL).render(**globals()))